### Importing modules

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)

import scipy
from pathlib import Path
from os.path import join as pjoin

import matplotlib.pyplot as plt
import matplotlib as mpl

new_rc_params = {'text.usetex': False,
"svg.fonttype": 'none'
}
mpl.rcParams.update(new_rc_params)
plt.rcParams['font.sans-serif'] = ['Arial'] 
plt.rcParams['font.size'] = 12

import chiCa.chiCa as chiCa
from chiCa.chiCa.visualization_utils import separate_axes
import spks

%matplotlib widget
%load_ext autoreload
%autoreload 2

### Load data

#### Loading raw nidaq and behavior data and preprocessing it

In [2]:
from utils import load_sync_data, process_port_events, process_trial_data

animal = 'GRB006'
session = '20240723_142451'
sessionpath = Path(f'/Volumes/grb_ephys/data/GRB006/{session}')

corrected_onsets, corrected_offsets, t, srate, analog_signal = load_sync_data(sessionpath)
trial_starts, port_events = process_port_events(corrected_onsets, corrected_offsets, srate)
behavior_data, trial_ts = process_trial_data(sessionpath, trial_starts, t, srate, analog_signal, port_events, animal, session)

Loading nisync data...


ValueError: x and y arrays must be equal in length along interpolation axis.

#### Loading and filtering KS results

In [ ]:
from utils import get_good_units, get_cluster_spike_times

kilosort_path = Path(f'{sessionpath}/kilosort2.5/imec0/')
sc = np.load(pjoin(kilosort_path, 'spike_clusters.npy')) #KS clusters
ss = np.load(pjoin(kilosort_path, 'spike_times.npy')) #KS spikes (in samples)
st = ss/srate #conversion from spike samples to spike times

clu =  spks.clusters.Clusters(folder = kilosort_path, get_waveforms=False, get_metrics=True, load_template_features=True)

good_units_mask, n_units = get_good_units(clusters_obj = clu, spike_clusters = sc)
print(f'# good units: {n_units}')

#### Saving preprocessed neural and behavioral data locally

In [5]:
save_dir = Path('processed_data') / animal / session
save_dir.mkdir(parents=True, exist_ok=True)

# Convert to object array before saving
spike_times_per_unit = get_cluster_spike_times(spike_times = st, spike_clusters = sc, good_unit_ids = good_units_mask)
spike_times_array = np.array(spike_times_per_unit, dtype=object)

# Save spike times and trial data
np.save(save_dir / 'spike_times_per_unit.npy', spike_times_array, allow_pickle=True)
trial_ts.to_pickle(save_dir / 'trial_ts.pkl')

#### Loading data for one session locally

In [2]:
save_dir = Path('processed_data')
animal = 'GRB006'  # example animal
session = '20240723_142451'  # example session

data_dir = save_dir / animal / session
trial_ts = pd.read_pickle(data_dir / 'trial_ts.pkl')
spike_times_per_unit = np.load(data_dir / 'spike_times_per_unit.npy', allow_pickle=True)